## 0. PSEUDOCODE / OVERVIEW

##### Before this script:
Summed the MapSPAM crop rasters for each year. Code is available at:
<br> https://github.com/gracedoherty/Misc/blob/main/MapSPAM%20batch%20sums.ipynb

<br> Manually renamed the Value of Production rasters from 2005 and 2010 to match naming conventions of the rest. (The VoP files for these years did not need to be summed in the previous script and still had their MapSPAM source file names.)

##### Prep data
Reproject MapSPAM to equal area.

##### Mask to area of interest
Because we are using the clunky tif-xyz-df-gdf zonal stats method, it will reduce processing time to first mask out pixels not in the region of interest.

##### Zonal statistics
Aggregation method: sum.

##### Change over time
Calculate the percent change in agro indicators between the four years of study (2000,2005,2010,2017).

## 1. PREPARE WORKSPACE

### 1.1 Load all packages.

In [1]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

In [2]:
ProjectFolder = os.getcwd()
print(ProjectFolder)

LZFolder = 'Q:\GIS\povertyequity\PTI_Sahel\LivelihoodZones_forPython'
print(LZFolder)

LZPath = os.path.join(LZFolder, 'LZ.gpkg')
print(LZPath)

Q:\GIS\povertyequity\PTI_Sahel\Agriculture
Q:\GIS\povertyequity\PTI_Sahel\LivelihoodZones_forPython
Q:\GIS\povertyequity\PTI_Sahel\LivelihoodZones_forPython\LZ.gpkg


### 1.2 User-defined functions.

In [3]:
def rioStats(InRasterPath, Band = 1):
    out = rasterio.open(InRasterPath)
    stats = []
    band = out.read(Band)
    stats.append({
        'raster': out.name,
        'bands': out.count,
        'data type': out.dtypes,
        'no data value': out.nodatavals,
        'width': out.width,
        'height': out.height,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})
    print("\n", stats)
    
    out = band = None

In [4]:
def MaskByZone(MaskPath, SourceFolder, DestFolder, SourceList = None,
               MaskLayerName = None, dstSRS = 'ESRI:102022'):
    """
    Reduces the size of a raster's valid data cells to vector areas of interest.
    This is useful if the raster data needs to be vectorized later to save space.
    
    The script prepares the vector zones as a list of geometries in the desired
    spatial reference system, then warps each raster in the specified source
    folder to the same SRS. Masking in rasterio then reclassifies any raster cells
    falling outside of a mask polygon as NoData.
    """
    
    ProjSRS = osr.SpatialReference()
    ProjSRS.SetFromUserInput(dstSRS)
    ProjWarp = gdal.WarpOptions(dstSRS = dstSRS)
    
    if SourceList is not None:
        SourceFiles = SourceList
    else:
        SourceFiles = []
        SourceFiles = SourceFiles + [i for i in os.listdir(''.join([SourceFolder, r'/'])) if i.endswith('tif')]
        print(SourceFiles)

    
    ### 1. ASSIGN SPATIAL REFERENCE SYSTEM OF VECTOR MASK AND LOAD GEOMETRIES
    Vector = gpd.read_file(filename=MaskPath, layer=MaskLayerName)
    if Vector.crs != dstSRS:
        if MaskLayerName == None:
            MaskPath = MaskPath + '_temp'
        else:
            MaskLayerName = MaskLayerName + '_temp'
        Vector.to_crs(dstSRS).to_file(filename=MaskPath, layer=MaskLayerName)
    Vector = None # We're reloading the geometries with fiona
    
    with fiona.open(MaskPath, mode="r", layer=MaskLayerName) as Vector:
        MaskGeom = [feature["geometry"] for feature in Vector] # Identify the bounding areas of the mask.
    
    
    ### 2. PREPARE DESTINATION FILES
    for FileName in SourceFiles:
    
        InputRasterPath = os.path.join(ProjectFolder, SourceFolder, FileName)

        TempOutputName = 'Temp_' + FileName
        TempOutputPath = os.path.join(ProjectFolder, DestFolder, TempOutputName)
        FinalOutputName = 'Msk_' + FileName
        FinalOutputPath = os.path.join(ProjectFolder, DestFolder, FinalOutputName)

    ### 3. ASSIGN SPATIAL REFERENCE SYSTEM OF RASTER(S)
        InputRasterObject = gdal.Open(InputRasterPath)
        SourceSRS = osr.SpatialReference(wkt=InputRasterObject.GetProjection())
        print('Source projection: ', SourceSRS.GetAttrValue('projcs'))
        print('Destination projection: ', ProjSRS.GetAttrValue('projcs'))

        if SourceSRS.GetAttrValue('projcs') != ProjSRS.GetAttrValue('projcs'):
            Warp = gdal.Warp(TempOutputPath, # Where to store the warped raster
                         InputRasterObject, # Which raster to warp
                         format='GTiff', 
                         options=ProjWarp) # Reproject to Africa Albers Equal Area Conic
            print('Finished gdal.Warp() for %s. %s \n' % (FileName, time.ctime()))

            Warp = None # Close the files
        else:
            pass
        InputRasterObject = None
        
    ### 4. RECLASSIFY AS NODATA IF OUTSIDE OF SETTLEMENT BUFFER ZONE.
        if exists(TempOutputPath):
            NewInputPath = TempOutputPath 
            print("We warped the data, so we'll use that file for next step.")
        else:
            NewInputPath = InputRasterPath 
            print("We skipped the warp, so we continue to use the source file.")

        with rasterio.open(NewInputPath) as InputRasterObject:
            MaskedOutputRaster, OutTransform = rasterio.mask.mask(
                InputRasterObject, MaskGeom, crop=True) # Anything outside the mask is reclassed to the raster's NoData value.
            OutMetaData = InputRasterObject.meta.copy()
        print('Finished rasterio.mask.mask() for %s. %s \n' % (FileName, time.ctime()))

        OutMetaData.update({"driver": "GTiff",
                         "height": MaskedOutputRaster.shape[1],
                         "width": MaskedOutputRaster.shape[2],
                         "transform": OutTransform})

        with rasterio.open(FinalOutputPath, "w", **OutMetaData) as dest:
            dest.write(MaskedOutputRaster)
        print('Written to file. %s \n' % time.ctime())
        InputRasterObject = None

        if exists(TempOutputPath):
            try:  # Finally, remove the intermediate file from disk
                os.remove(TempOutputPath)
            except OSError:
                pass
            print('Removed intermediate file. %s \n' % time.ctime())
        else:
            pass


    print('\n \n Finished all years in list. %s' % time.ctime())

In [23]:
def BatchZonal_Step1(SourceFolder, Zones, 
                    DestFolder=None, 
                    CRS = 'ESRI:102022', 
                    JoinField = 'ID',
                    FilesList = None,
                    IndicatorName = None):
    """
    Normally, we would use numpy to generate a point gdf from the raster's matrix. 
    However, I was running into a lot of memory errors with that method.
    This method uses some extra steps: tif to xyz to df to gdf. But it saves to file
    and deletes intermediate files along the way, circumventing memory issues.
    
    Run MaskByZone() prior to reduce the raster to only your area(s) of interest.
    
    """
    if DestFolder is None:
        DestFolder = SourceFolder.copy()

    if FilesList is None:
        FilesList = [i for i in os.listdir(SourceFolder) if i.endswith('.tif')]
    print(FilesList)
    
    
    for FileName in FilesList:
        
        if IndicatorName is None:
            Indicator = FileName.replace('.tif', '')
        else:
            Indicator = IndicatorName.copy()

    ### SPECIFIC TO MAPSPAM TO ADM SCRIPT ###
        try: 
            Indicator = Indicator.replace('Msk_', '')
        except:
            pass
        try:
            Indicator = Indicator.replace('_20', '')
        except:
            pass
        ### ###
        
    ### STEP 1: TIF TO XYZ ###
        print('Loading data for %s. %s \n' % (FileName, time.ctime()))
        
        InputRasterPath = os.path.join(os.getcwd(), SourceFolder, FileName)
        InputRasterObject = gdal.Open(InputRasterPath)
        XYZOutputPath = DestFolder + r'/{}'.format(
            FileName.replace('.tif', '.xyz')) # New file path will be the same as original, but .tif is replaced with .xyz

        # Create an .xyz version of the .tif
        if exists(XYZOutputPath):
            print("Already created xyz file.")
        else:
            print("Creating XYZ (gdal.Translate()).")
            XYZ = gdal.Translate(XYZOutputPath, # Specify a destination path
                                 InputRasterObject, # Input is the masked .tif file
                                 format='XYZ', 
                                 creationOptions=["ADD_HEADER_LINE=YES"])
            print('Finished gdal.Translate() for file: %s. %s \n' % (FileName, time.ctime()))
            XYZ = None # Reload XYZ as a point geodataframe

        InputRasterObject = None


    ### STEP 2: GENERATE GEODATAFRAME WITH JOIN FIELD ###
        InputXYZ = pd.read_table(XYZOutputPath, delim_whitespace=True)
        
        InputXYZ.Z = pd.to_numeric(InputXYZ.Z, errors='coerce').fillna(-1).astype(np.int64) # Subsetting doesn't work if there are some values that got a str designation.
        InputXYZ = InputXYZ.loc[InputXYZ['Z'] > -1] # Subset to only the features that have a valid value.
            
        print('Loaded XYZ file as a pandas dataframe. %s \n' % time.ctime())
        ValObject = gpd.GeoDataFrame(InputXYZ,
                                     geometry = gpd.points_from_xy(InputXYZ['X'], InputXYZ['Y']),
                                     crs = CRS)
        print('Created geodataframe from non-NoData points. %s \n' % time.ctime())
        del InputXYZ

        # Sjoin_nearest: No need to group by ADM this time. 
        ValObject_withID = pd.DataFrame(gpd.sjoin_nearest(ValObject[['Z', 'geometry']], 
                                        Zones, 
                                        how='left')).drop(columns='geometry') # No need for max_distance parameter this time. We've already narrowed down to nearby raster cells.

        print('\nJoined zone ID onto vectorized raster cells. %s \n' % time.ctime())
        print(ValObject_withID.sample(10))
        del ValObject

        ValObject_withID.to_csv(''.join([DestFolder, r'/', FileName.replace('.tif', '.csv')]))
        print('\nExported as table. %s \n' % time.ctime())
        
    ### CLEAN UP ###
        if exists(XYZOutputPath):
            try:  # Finally, remove the intermediate file from disk
                os.remove(XYZOutputPath)
                print('Removed intermediate file. %s \n' % time.ctime())
            except OSError:
                pass
        else:
            pass

    print('Finished prepping for zonal stats (group-by summaries). %s' % time.ctime())

In [29]:
def BatchZonal_Step2(SourceFolder, Zones, 
                    DestFolder=None,  
                    JoinField = 'ID',
                    StatsWanted = ['count', 'sum', 'mean', 'max', 'min'],
                    FilesList = None,
                    IndicatorName = None,
                    Prefix = '',
                    Suffix = ''):

    if DestFolder is None:
        DestFolder = SourceFolder.copy()

    if FilesList is None:
        FilesList = [i for i in os.listdir(SourceFolder) if i.endswith('.csv')]
    print(FilesList)
    
    for FileName in FilesList:

        if IndicatorName is None:
            Indicator = FileName.replace('.csv', '')
        else:
            Indicator = IndicatorName.copy()

        ### SPECIFIC TO MAPSPAM TO ADM SCRIPT ###
        try: 
            Indicator = Indicator.replace('Msk_', '')
        except:
            pass
        try:
            Indicator = Indicator.replace('_20', '')
        except:
            pass
        ### ###
        

        AllSummaries = pd.DataFrame(Zones).drop(columns='geometry')[[JoinField]]
        print(AllSummaries)

    ### AGGREGATE BY SETTLEMENT AND MERGE ONTO SUMMARIES TABLE ###
        InDF = pd.read_csv(os.path.join(SourceFolder, FileName))[['Z', JoinField]]
        GroupedVals = InDF[InDF['Z'].notna()].groupby(JoinField, as_index=False)
        
        if 'count' in StatsWanted:
            VariableName = ''.join([Indicator, '_ct'])
            AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
        if 'sum' in StatsWanted:
            VariableName = ''.join([Indicator, '_sum'])
            AllSummaries = AllSummaries.merge(GroupedVals.sum().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
        if 'mean' in StatsWanted:
            VariableName = ''.join([Indicator, '_avg'])
            AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
        if 'max' in StatsWanted:
            VariableName = ''.join([Indicator, '_max'])
            AllSummaries = AllSummaries.merge(GroupedVals.max().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
        if 'min' in StatsWanted:
            VariableName = ''.join([Indicator, '_min'])
            AllSummaries = AllSummaries.merge(GroupedVals.min().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
        print('\nDesired aggregation methods applied to zone level, file: %s. %s \n' % (FileName, time.ctime()))
            
        # Save results for this file.
        AllSummaries.to_csv(os.path.join(DestFolder, ''.join([Prefix, Indicator, Suffix, '.csv'])))
        print(AllSummaries.sample(10))

print('\n\nFinished. All rasters from source list aggregated to the zone they fall into. %s' % time.ctime())



Finished. All rasters from source list aggregated to the zone they fall into. Wed May  3 18:48:16 2023


## 2. LOAD DATA

##### ADMs

In [6]:
# The ADMs in the LZ.gpkg are already in our preferred projection.
ADM3, ADM2, ADM1 = gpd.read_file(LZPath, layer='ADM3'), gpd.read_file(LZPath, layer='ADM2'), gpd.read_file(LZPath, layer='ADM1')
ADM3['FID'], ADM2['FID'], ADM1['FID'] = ADM3.index, ADM2.index, ADM1.index
ADM3.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   admin3Name  1433 non-null   object  
 1   admin3Pcod  1433 non-null   object  
 2   admin2Pcod  1433 non-null   object  
 3   admin1Pcod  1433 non-null   object  
 4   admin0Pcod  1433 non-null   object  
 5   ADM3_CODE   1433 non-null   object  
 6   ADM2_CODE   1433 non-null   object  
 7   ADM1_CODE   1433 non-null   object  
 8   geometry    1433 non-null   geometry
 9   FID         1433 non-null   int64   
dtypes: geometry(1), int64(1), object(8)
memory usage: 112.1+ KB


In [19]:
ADMlist = [ADM1, ADM2, ADM3]
ADMlist[2].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   admin3Name  1433 non-null   object  
 1   admin3Pcod  1433 non-null   object  
 2   admin2Pcod  1433 non-null   object  
 3   admin1Pcod  1433 non-null   object  
 4   admin0Pcod  1433 non-null   object  
 5   ADM3_CODE   1433 non-null   object  
 6   ADM2_CODE   1433 non-null   object  
 7   ADM1_CODE   1433 non-null   object  
 8   geometry    1433 non-null   geometry
 9   FID         1433 non-null   int64   
dtypes: geometry(1), int64(1), object(8)
memory usage: 112.1+ KB


##### Agro rasters

In [8]:
SourceFiles = []
SourceFiles = SourceFiles + [i for i in os.listdir('Global_Agro_Sums') if i.endswith('tif')]
SourceFiles

['HarvArea_2000_allTech.tif',
 'HarvArea_2000_Irrigated.tif',
 'HarvArea_2005_allTech.tif',
 'HarvArea_2005_Irrigated.tif',
 'HarvArea_2010_allTech.tif',
 'HarvArea_2010_Irrigated.tif',
 'HarvArea_2017_allTech.tif',
 'HarvArea_2017_Irrigated.tif',
 'PhysArea_2000_allTech.tif',
 'PhysArea_2000_Irrigated.tif',
 'PhysArea_2005_allTech.tif',
 'PhysArea_2005_Irrigated.tif',
 'PhysArea_2010_allTech.tif',
 'PhysArea_2010_Irrigated.tif',
 'PhysArea_2017_allTech.tif',
 'PhysArea_2017_Irrigated.tif',
 'Val_2005_allTech.tif',
 'Val_2005_Irrigated.tif',
 'Val_2010_allTech.tif',
 'Val_2010_Irrigated.tif',
 'Val_2017_allTech.tif',
 'Val_2017_Irrigated.tif']

In [9]:
for File in SourceFiles:
    Path = os.path.join('Global_Agro_Sums', File)
    rioStats(Path)


 [{'raster': 'Global_Agro_Sums\\HarvArea_2000_allTech.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 104.58319, 'median': -1.0, 'max': 20909.902}]

 [{'raster': 'Global_Agro_Sums\\HarvArea_2000_Irrigated.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 25.266144, 'median': -1.0, 'max': 16489.701}]

 [{'raster': 'Global_Agro_Sums\\HarvArea_2005_allTech.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-3.4028234663852886e+38,), 'width': 4320, 'height': 1853, 'min': nan, 'mean': nan, 'median': nan, 'max': nan}]

 [{'raster': 'Global_Agro_Sums\\HarvArea_2005_Irrigated.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-3.4028234663852886e+38,), 'width': 4320, 'height': 1853, 'min': nan, 'mean': nan, 'median': nan, 'max': nan}]

 [{'raster': 'Global_Agro_Sums\\HarvArea_2010_allTech.tif', 'bands': 1, 'data type': ('fl

C:\Users\grace\miniconda3\envs\geo\lib\site-packages\numpy\core\_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)



 [{'raster': 'Global_Agro_Sums\\Val_2010_allTech.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 159637.23, 'median': -1.0, 'max': 128835730.0}]

 [{'raster': 'Global_Agro_Sums\\Val_2010_Irrigated.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 55472.035, 'median': -1.0, 'max': 79640680.0}]

 [{'raster': 'Global_Agro_Sums\\Val_2017_allTech.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 15054.96, 'median': -1.0, 'max': 81860930.0}]

 [{'raster': 'Global_Agro_Sums\\Val_2017_Irrigated.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 956.40625, 'median': -1.0, 'max': 80450770.0}]


## 3. MASK TO AREA OF INTEREST

This also reprojects anything that needs to be into the preferred spatial reference.

In [ ]:
# for File in SourceFiles:
#     inRaster = gdal.Open(os.path.join('Global_Agro_Sums', File))
#     outPath = os.path.join('IntermediateFiles', File.replace('.tif', 'EqArea.tif'))
#     warp = gdal.Warp(outPath,inRaster,dstSRS='ESRI:102022')
#     warp = None # Closes the files

In [ ]:
MaskByZone(MaskPath=LZPath, SourceFolder = 'Global_Agro_Sums', DestFolder='IntermediateFiles', 
           SourceList = SourceFiles, MaskLayerName = 'ADM1', dstSRS = 'ESRI:102022')

## 4. ZONAL STATISTICS

In [24]:
BatchZonal_Step1(SourceFolder='IntermediateFiles', 
                 Zones=ADMlist[2], 
                 DestFolder='IntermediateFiles', 
                 CRS = 'ESRI:102022', 
                 JoinField = 'ADM3_CODE', # The function also retains ADM1 and 2 codes in the resulting csv.
                 FilesList = None,
                 IndicatorName = None)

['Msk_HarvArea_2000_allTech.tif', 'Msk_HarvArea_2000_Irrigated.tif', 'Msk_HarvArea_2005_allTech.tif', 'Msk_HarvArea_2005_Irrigated.tif', 'Msk_HarvArea_2010_allTech.tif', 'Msk_HarvArea_2010_Irrigated.tif', 'Msk_HarvArea_2017_allTech.tif', 'Msk_HarvArea_2017_Irrigated.tif', 'Msk_PhysArea_2000_allTech.tif', 'Msk_PhysArea_2000_Irrigated.tif', 'Msk_PhysArea_2005_allTech.tif', 'Msk_PhysArea_2005_Irrigated.tif', 'Msk_PhysArea_2010_allTech.tif', 'Msk_PhysArea_2010_Irrigated.tif', 'Msk_PhysArea_2017_allTech.tif', 'Msk_PhysArea_2017_Irrigated.tif', 'Msk_Val_2005_allTech.tif', 'Msk_Val_2005_Irrigated.tif', 'Msk_Val_2010_allTech.tif', 'Msk_Val_2010_Irrigated.tif', 'Msk_Val_2017_allTech.tif', 'Msk_Val_2017_Irrigated.tif']
Loading data for Msk_HarvArea_2000_allTech.tif. Wed May  3 18:35:41 2023 

Already created xyz file.
Loaded XYZ file as a pandas dataframe. Wed May  3 18:35:41 2023 

Created geodataframe from non-NoData points. Wed May  3 18:35:41 2023 


Joined zone ID onto vectorized raster cel

Finished gdal.Translate() for file: Msk_HarvArea_2010_allTech.tif. Wed May  3 18:36:17 2023 

Loaded XYZ file as a pandas dataframe. Wed May  3 18:36:17 2023 

Created geodataframe from non-NoData points. Wed May  3 18:36:17 2023 


Joined zone ID onto vectorized raster cells. Wed May  3 18:36:28 2023 

          Z  index_right         admin3Name    admin3Pcod admin2Pcod  \
58755   135         1217              Sanam  NER006001002  NER006001   
61820   581          314         Nassoumbou    Nassoumbou     BF5603   
58157   206          620              Balle      ML523113      ML523   
84895   909         1387  Haraze Mangueigne       TD15090    TD15090   
76495   274          592     Ouelessebougou       ML52099      ML520   
75963  2331         1423    Koukou-Angarana       TD21022    TD21022   
68953  1283           53             Tougan        Tougan     BF4606   
93794    14         1365   Mandoul Oriental       TD10029    TD10029   
63054  1008          384            Diallan    


Joined zone ID onto vectorized raster cells. Wed May  3 18:37:11 2023 

          Z  index_right         admin3Name    admin3Pcod admin2Pcod  \
78584   215         1331           Baguirmi       TD03009    TD03009   
84898    58         1387  Haraze Mangueigne       TD15090    TD15090   
72495  6532          931           MPessoba      ML842235      ML842   
85884   291          876           Manakoro      ML839219      ML839   
60345  1541          426           Ambidedi       ML30939      ML309   
58795  6205         1136        Bader Goula  NER004003003  NER004003   
82638   137         1338       Bahr Signaka       TD04052    TD04052   
71434   344         1341             Ngoura       TD05049    TD05049   
63438   755         1381              Ouara       TD14013    TD14013   
63597  1163          248              Sollé         Sollé     BF5401   

      admin1Pcod admin0Pcod ADM3_CODE ADM2_CODE ADM1_CODE   FID  
78584       TD03         TD  TD030101    TD0301      TD03  1331  
84

Finished gdal.Translate() for file: Msk_PhysArea_2010_Irrigated.tif. Wed May  3 18:37:48 2023 

Loaded XYZ file as a pandas dataframe. Wed May  3 18:37:48 2023 

Created geodataframe from non-NoData points. Wed May  3 18:37:48 2023 


Joined zone ID onto vectorized raster cells. Wed May  3 18:37:59 2023 

       Z  index_right    admin3Name    admin3Pcod admin2Pcod admin1Pcod  \
73255  0         1339      Mangalmé       TD04089    TD04089       TD04   
65434  0         1257      Kourteye  NER006012004  NER006012     NER006   
78125  0         1331      Baguirmi       TD03009    TD03009       TD03   
69431  0          115      Tougouri      Tougouri     BF4902       BF49   
70339  0          199  Bartiébougou  Bartiébougou     BF5203       BF52   
63713  0         1130       Kornaka  NER004003008  NER004003     NER004   
84423  0         1338  Bahr Signaka       TD04052    TD04052       TD04   
63134  0          632    Bandiagara      ML624119      ML624        ML6   
73729  0         1

Finished gdal.Translate() for file: Msk_Val_2010_allTech.tif. Wed May  3 18:38:35 2023 

Loaded XYZ file as a pandas dataframe. Wed May  3 18:38:35 2023 

Created geodataframe from non-NoData points. Wed May  3 18:38:35 2023 


Joined zone ID onto vectorized raster cells. Wed May  3 18:38:46 2023 

            Z  index_right         admin3Name    admin3Pcod admin2Pcod  \
76608  160714          210           Partiaga      Partiaga     BF5205   
57709  136567          621              Balle      ML523113      ML523   
90226  232248         1380            Lac Iro       TD13081    TD13081   
81757   82585         1385         Bahr Azoum       TD15001    TD15001   
81770    4794         1387  Haraze Mangueigne       TD15090    TD15090   
67592  191729          843               Tene      ML737206      ML737   
61910  624641         1184              Bouza  NER005004003  NER005004   
65772  651881          530         Toukouroba       ML51785      ML517   
82740  403103          985       G

In [30]:
# Note: All SPAM summary files have NoData values that are -1 or less.
# Therefore there is no need to tweak the Batch function, which already includes only values more than -1.

for ADM in ADMlist:
    if 'ADM3_CODE' in ADM.columns:
        JoinID = 'ADM3_CODE'
    elif 'ADM2_CODE' in ADM.columns:
        JoinID = 'ADM2_CODE'
    else:
        JoinID = 'ADM1_CODE'
    
    BatchZonal_Step2(SourceFolder='IntermediateFiles', 
                     Zones=ADM, 
                     DestFolder='Summaries', 
                     JoinField = JoinID,
                     StatsWanted = ['count', 'sum', 'mean', 'max', 'min'],
                     FilesList = None,
                     IndicatorName = None,
                     Suffix =''.join(['_', JoinID.replace('_CODE', '')]))

['Msk_HarvArea_2000_allTech.csv', 'Msk_HarvArea_2000_Irrigated.csv', 'Msk_HarvArea_2005_allTech.csv', 'Msk_HarvArea_2005_Irrigated.csv', 'Msk_HarvArea_2010_allTech.csv', 'Msk_HarvArea_2010_Irrigated.csv', 'Msk_HarvArea_2017_allTech.csv', 'Msk_HarvArea_2017_Irrigated.csv', 'Msk_PhysArea_2000_allTech.csv', 'Msk_PhysArea_2000_Irrigated.csv', 'Msk_PhysArea_2005_allTech.csv', 'Msk_PhysArea_2005_Irrigated.csv', 'Msk_PhysArea_2010_allTech.csv', 'Msk_PhysArea_2010_Irrigated.csv', 'Msk_PhysArea_2017_allTech.csv', 'Msk_PhysArea_2017_Irrigated.csv', 'Msk_Val_2005_allTech.csv', 'Msk_Val_2005_Irrigated.csv', 'Msk_Val_2010_allTech.csv', 'Msk_Val_2010_Irrigated.csv', 'Msk_Val_2017_allTech.csv', 'Msk_Val_2017_Irrigated.csv']
   ADM1_CODE
0       BF01
1       BF02
2       BF03
3       BF04
4       BF05
5       BF06
6       BF07
7       BF08
8       BF09
9       BF10
10      BF11
11      BF12
12      BF13
13      ML01
14      ML02
15      ML03
16      ML04
17      ML05
18      ML06
19      ML07
20      


Desired aggregation methods applied to zone level, file: Msk_HarvArea_2010_allTech.csv. Wed May  3 18:48:25 2023 

   ADM1_CODE  HarvArea10_allTech_ct  HarvArea10_allTech_sum  \
30      TD01                    422                  410646   
21      ML09                    621                  213986   
33      TD04                    694                  381532   
14      ML02                    520                   89272   
45      TD16                    199                  327667   
38      TD09                    223                  249374   
37      TD08                    125                  269700   
32      TD03                    534                  638716   
43      TD14                    446                  514186   
19      ML07                    635                 1366032   

    HarvArea10_allTech_avg  HarvArea10_allTech_max  HarvArea10_allTech_min  
30              973.094787                    3345                       6  
21              344.582931          


Desired aggregation methods applied to zone level, file: Msk_PhysArea_2000_Irrigated.csv. Wed May  3 18:48:25 2023 

   ADM1_CODE  PhysArea00_Irrigated_ct  PhysArea00_Irrigated_sum  \
2       BF03                    215.0                    6593.0   
29      NE08                      9.0                      95.0   
30      TD01                    770.0                     290.0   
12      BF13                    166.0                     221.0   
49      TD20                      NaN                       NaN   
33      TD04                    801.0                     142.0   
1       BF02                    479.0                    4423.0   
21      ML09                    541.0                   50100.0   
15      ML03                   1359.0                    3179.0   
4       BF05                    272.0                     882.0   

    PhysArea00_Irrigated_avg  PhysArea00_Irrigated_max  \
2                  30.665116                    1368.0   
29                 10.555556

   ADM1_CODE  PhysArea17_allTech_ct  PhysArea17_allTech_sum  \
37      TD08                  125.0                267129.0   
20      ML08                  976.0               2571115.0   
32      TD03                  518.0                500961.0   
19      ML07                  744.0               2101045.0   
39      TD10                  240.0                135683.0   
50      TD21                  342.0                423327.0   
43      TD14                  433.0                434719.0   
46      TD17                  306.0                182632.0   
33      TD04                  613.0                299117.0   
11      BF12                  452.0                424135.0   

    PhysArea17_allTech_avg  PhysArea17_allTech_max  PhysArea17_allTech_min  
37             2137.032000                  3148.0                   300.0  
20             2634.339139                  8444.0                     1.0  
32              967.106178                  4800.0                     1.0 


Desired aggregation methods applied to zone level, file: Msk_Val_2017_allTech.csv. Wed May  3 18:48:25 2023 

   ADM1_CODE  Val17_allTech_ct  Val17_allTech_sum  Val17_allTech_avg  \
41      TD12             180.0        116288479.0       6.460471e+05   
18      ML06              82.0        114971675.0       1.402094e+06   
10      BF11             121.0         87295051.0       7.214467e+05   
46      TD17             306.0         48089125.0       1.571540e+05   
21      ML09             110.0        271852630.0       2.471388e+06   
43      TD14             433.0        160202181.0       3.699819e+05   
44      TD15             380.0         90060582.0       2.370015e+05   
36      TD07             172.0         34938330.0       2.031298e+05   
39      TD10             240.0         92473412.0       3.853059e+05   
35      TD06             139.0          7294859.0       5.248100e+04   

    Val17_allTech_max  Val17_allTech_min  
41          3623131.0             3345.0  
18        

...


Desired aggregation methods applied to zone level, file: Msk_Val_2010_allTech.csv. Wed May  3 18:48:26 2023 

    ADM2_CODE  Val10_allTech_ct  Val10_allTech_sum  Val10_allTech_avg  \
83     ML0801               9.0          9928849.0       1.103205e+06   
181    TD0405              99.0          6997156.0       7.067834e+04   
169    TD0201               9.0          2071380.0       2.301533e+05   
16     BF0602              67.0         30915332.0       4.614229e+05   
180    TD0404              83.0          2547291.0       3.069025e+04   
178    TD0402             125.0         22934773.0       1.834782e+05   
212    TD1103              38.0         24445248.0       6.432960e+05   
6      BF0206              83.0         32232089.0       3.883384e+05   
225    TD1403              33.0         10479860.0       3.175715e+05   
213    TD1104              47.0         23827647.0       5.069712e+05   

     Val10_allTech_max  Val10_allTech_min  
83           1741227.0           439904.0


Desired aggregation methods applied to zone level, file: Msk_HarvArea_2000_Irrigated.csv. Wed May  3 18:48:27 2023 

     ADM3_CODE  HarvArea00_Irrigated_ct  HarvArea00_Irrigated_sum  \
683   ML060409                      8.0                       0.0   
572   ML050410                      1.0                       0.0   
644   ML060115                      2.0                       0.0   
522   ML050101                      8.0                       0.0   
392   ML030112                      5.0                       0.0   
825   ML070607                      3.0                       0.0   
650   ML060121                      5.0                       0.0   
273   BF100402                      7.0                       0.0   
1315  NE071105                      1.0                       0.0   
465   ML030517                     10.0                       0.0   

      HarvArea00_Irrigated_avg  HarvArea00_Irrigated_max  \
683                        0.0                       0.0   
57


Desired aggregation methods applied to zone level, file: Msk_HarvArea_2017_allTech.csv. Wed May  3 18:48:27 2023 

     ADM3_CODE  HarvArea17_allTech_ct  HarvArea17_allTech_sum  \
1416  TD200401                    NaN                     NaN   
1228  NE060702                   27.0                207562.0   
1330  TD020401                    NaN                     NaN   
33    BF020401                   17.0                 62355.0   
282   BF110106                   10.0                 36928.0   
16    BF020110                    6.0                 15986.0   
586   ML050424                    4.0                  3352.0   
1119  NE030702                   16.0                208454.0   
1277  NE070406                   24.0                 80881.0   
1269  NE070302                   16.0                133300.0   

      HarvArea17_allTech_avg  HarvArea17_allTech_max  HarvArea17_allTech_min  
1416                     NaN                     NaN                     NaN  
1228      

     ADM3_CODE  PhysArea17_allTech_ct  PhysArea17_allTech_sum  \
812   ML070523                    3.0                 10123.0   
371   ML020303                    NaN                     NaN   
612   ML050603                    9.0                 11504.0   
1149  NE040602                   10.0                 51383.0   
124   BF050307                    5.0                  7408.0   
760   ML070211                    5.0                 16577.0   
68    BF030206                    8.0                 14259.0   
1293  NE070607                    2.0                 11039.0   
63    BF030201                    6.0                 28019.0   
290   BF110301                    5.0                  7864.0   

      PhysArea17_allTech_avg  PhysArea17_allTech_max  PhysArea17_allTech_min  
812              3374.333333                  3928.0                  2394.0  
371                      NaN                     NaN                     NaN  
612              1278.222222                  3


Desired aggregation methods applied to zone level, file: Msk_Val_2010_Irrigated.csv. Wed May  3 18:48:27 2023 

     ADM3_CODE  Val10_Irrigated_ct  Val10_Irrigated_sum  Val10_Irrigated_avg  \
934   ML080531                 1.0                  0.0         0.000000e+00   
672   ML060310                 7.0            3933662.0         5.619517e+05   
678   ML060404                 7.0                  0.0         0.000000e+00   
700   ML060511                25.0             770462.0         3.081848e+04   
211   BF080507                11.0                  0.0         0.000000e+00   
1087  NE030106                 2.0                  0.0         0.000000e+00   
86    BF040203                 5.0                  0.0         0.000000e+00   
782   ML070405                 4.0            6944629.0         1.736157e+06   
1217  NE060102                56.0                  0.0         0.000000e+00   
682   ML060408                 4.0                  0.0         0.000000e+00   

      

## 5. Change over time

## 6. Save to file.